# Knihovna OpenPyXL

V této lekci se naučíme pracovat s Excel soubory pomocí knihovny `openpyxl`. Tato knihovna umožňuje vytvářet, otevírat a upravovat Excel soubory ve formátu `.xlsx`.

## Instalace a import knihovny

Pokud nemáte knihovnu nainstalovanou, spusťte:
```
pip install openpyxl
```

In [1]:
import openpyxl

## Hello World - vytvoření nového Excel souboru

Základní kroky pro vytvoření souboru:
1. Vytvořit objekt `Workbook`
2. Vybrat aktivní list
3. Zapsat data do buněk
4. Uložit soubor

In [2]:
# Vytvoříme nový Workbook objekt
wb = openpyxl.Workbook()

In [3]:
wb

In [4]:
wb.active

<Worksheet "Sheet">

In [5]:
# Vybereme aktivní list
ws = wb.active

In [6]:
# Zapíšeme data do buňky A1
ws['A1'] = 'Hello world'

In [7]:
# Alternativní způsob zápisu do buňky
ws.cell(row=2, column=1, value='Druhý řádek')

<Cell 'Sheet'.A2>

In [8]:
# Uložíme soubor
wb.save('hello_world.xlsx')

### Otázka k zamyšlení
Jaký je rozdíl mezi zápisem `ws['A1'] = hodnota` a `ws.cell(row=1, column=1, value=hodnota)`?

## Čtení dat z existujícího souboru

Pro otevření existujícího souboru použijeme funkci `load_workbook()`.

In [9]:
# Otevřeme existující soubor
wb = openpyxl.load_workbook('hello_world.xlsx')

In [10]:
wb

In [11]:
# Zobrazíme seznam dostupných listů
wb.sheetnames

['Sheet']

In [12]:
wb['Sheet']

<Worksheet "Sheet">

In [13]:
# Vybereme konkrétní list podle jména
ws = wb['Sheet']

In [14]:
ws

<Worksheet "Sheet">

In [15]:
# Přečteme hodnotu z buňky A1
ws['A1'].value

'Hello world'

In [16]:
ws['A2'].value

'Druhý řádek'

In [17]:
ws['B4'].value

### Úloha: Oprav chybu v kódu

Následující kód obsahuje chybu. Najdi ji a oprav.

In [18]:
# OPRAV CHYBU:
wb = openpyxl.load_workbook('./hello_world.xlsx')
ws = wb.active
hodnota = ws['A1'].value  # Tady chybí něco...
hodnota

'Hello world'

## Práce s DataFrame

OpenPyXL nabízí funkci `dataframe_to_rows()` pro snadný převod DataFrame na řádky, které lze zapsat do Excelu.

In [19]:
import pandas as pd

In [20]:
# Vytvoříme DataFrame s informacemi o autech
df = pd.DataFrame(
    [['fiat 126p', 105, 630], 
     ['golf II', 148, 920], 
     ['tico', 143, 670]],
    columns=['model', 'max_speed', 'weight']
)

In [21]:
print(df)

       model  max_speed  weight
0  fiat 126p        105     630
1    golf II        148     920
2       tico        143     670


In [22]:
from openpyxl.utils.dataframe import dataframe_to_rows

In [23]:
dataframe_to_rows(df)

<generator object dataframe_to_rows at 0x000001B1DC8527A0>

In [24]:
# Zapíšeme DataFrame do Excelu
wb = openpyxl.Workbook()
ws = wb.active

In [25]:
for row in dataframe_to_rows(df):
    ws.append(row)

In [26]:
wb.save('cars.xls') # ulozilo indexy, aj prazdny riadok

In [27]:
# tu je nieco nechcene

In [28]:
for car in dataframe_to_rows(df, index=False, header=True):
    ws.append(car)  # Přidá řádek na konec listu

In [29]:
wb.save('cars.xlsx')

Parametry funkce `dataframe_to_rows()`:
- `df` - DataFrame, který chceme uložit
- `index` - zda chceme uložit index (výchozí: True)
- `header` - zda chceme uložit záhlaví sloupců (výchozí: True)

In [30]:
# odznova, nacisto
wb = openpyxl.Workbook()
ws = wb.active
for car in dataframe_to_rows(df, index=False, header=True):
    ws.append(car)  # Přidá řádek na konec listu
wb.save('cars.xlsx')

### Otázka k zamyšlení
Co se stane, když nastavíme `index=True`? Zkuste to a podívejte se na výsledný soubor.

## Filtry a zobrazení dat

OpenPyXL umožňuje definovat filtry přímo v Excel souboru. Filtry se však aplikují až při otevření souboru v Excelu.

In [31]:
# Příklad filtrace - zobrazí pouze 'tico'
wb = openpyxl.Workbook()
ws = wb.active

for car in dataframe_to_rows(df, index=False, header=True):
    ws.append(car)

In [32]:
wb.save('cars_filtered.xlsx')

In [33]:
# Nastavíme rozsah dat pro filtr
ws.auto_filter.ref = 'A1:C4'

In [34]:
# Přidáme filtr na první sloupec (index 0)
ws.auto_filter.add_filter_column(
    col_id=0,           # Sloupec, podle kterého filtrujeme
    vals=['tico']       # Hodnoty, které chceme zobrazit
)

# Skutočné skrytie riadkov
filter_hodnota = 'tico'
for row_idx in range(2, ws.max_row + 1):  # od 2, preskočí hlavičku
    cell_value = ws.cell(row=row_idx, column=1).value
    if cell_value != filter_hodnota:
        ws.row_dimensions[row_idx].hidden = True

In [35]:
wb.save('cars_filtered.xlsx')

## Vytváření nových listů

Pro vytvoření nového listu použijeme metodu `create_sheet()`.

In [36]:
wb = openpyxl.Workbook()

In [37]:
wb.active

<Worksheet "Sheet">

In [38]:
wb.sheetnames

['Sheet']

In [39]:
# Vytvoříme nový list s názvem a pozicí
ws1 = wb.create_sheet('Data', 0)      # První pozice

In [40]:
wb.sheetnames

['Data', 'Sheet']

In [41]:
ws2 = wb.create_sheet('Souhrn', 1)    # Druhá pozice

In [42]:
print(wb.sheetnames)

['Data', 'Souhrn', 'Sheet']


In [43]:
# Zapíšeme data do jednotlivých listů
ws1['A1'] = 'Toto je list Data'
ws2['A1'] = 'Toto je list Souhrn'

wb.save('vice_listu.xlsx')

### Úloha: Oprav chybu v kódu

Následující kód se snaží vytvořit nový list, ale obsahuje chybu.

In [44]:
# OPRAV CHYBU:
wb = openpyxl.Workbook()
ws = wb.create_sheet('MujList',1)  # Parametry jsou v nesprávném pořadí
ws['A1'] = 'Test'
wb.save('./test.xlsx')

In [45]:
wb.sheetnames

['Sheet', 'MujList']

---

# Cvičení

## Cvičení 1: Konverze souboru

Pomocí knihovny `openpyxl` převeďte soubor **product_prices_cleaned.csv** na Excel soubor.

In [46]:
# Váš kód zde:
df = pd.read_csv('../Data/product_prices_cleaned.csv', sep = ';')

In [47]:
wb = openpyxl.Workbook()
ws = wb.active

In [48]:
for row in dataframe_to_rows(df, header = True, index = False):
    ws.append(row)

In [49]:
wb.save('../Data/product_prices_cleaned.xlsx')

## Cvičení 2: Rozdělení dat do listů

Upravte předchozí cvičení tak, aby každá skupina produktů (`product_group_id`) byla v samostatném listu. Například `product_group_id` 1 by mělo být v listu s názvem `1`.

Použijte metodu:
```python
wb.create_sheet(name, index)
```
Kde:
- `name` - název listu (musí být řetězec)
- `index` - pozice listu v sešitu

In [50]:
df.columns

Index(['province', 'product_types', 'currency', 'product_group_id',
       'product_line', 'value', 'date', 'product'],
      dtype='object')

In [51]:
len(df['product_group_id'].unique())

4

In [52]:
df['product_group_id'].unique()

array([2, 4, 1, 3])

In [53]:
# Váš kód zde:
# rozdelim podla nazvu produktu
dfg = df.groupby(by = 'product_group_id')
dfg

In [54]:
wb = openpyxl.Workbook()

In [55]:
for g_id, _df in dfg:
    ws = wb.create_sheet(title = str(g_id), index = g_id)

In [56]:
wb.sheetnames

['Sheet', '1', '2', '3', '4']

In [57]:
wb = openpyxl.Workbook()

for (g_id, _df) in dfg:
    ws = wb.create_sheet(title = str(g_id), index = g_id)
    for row in dataframe_to_rows(_df, index = False, header = True):
        ws.append(row)

In [58]:
wb.sheetnames

['Sheet', '1', '2', '3', '4']

In [59]:
del wb['Sheet']

In [60]:
wb.sheetnames

['1', '2', '3', '4']

In [61]:
wb.save('../Data/product_prices_cleaned_groupped.xlsx')

In [62]:
# ak by produkty nemali ID 1, 2, 3, 4, tak hore kod nefunguje
wb = openpyxl.Workbook()

for idx, (g_id, _df) in enumerate(dfg):
    ws = wb.create_sheet(title = str(g_id), index = idx) 
    for row in dataframe_to_rows(_df, index = False, header = True):
        ws.append(row)
    

In [63]:
wb.sheetnames

['1', '2', '3', '4', 'Sheet']

In [64]:
del wb['Sheet']

## Cvičení 3: Automatizace pro analytiky

Tým analytiků připravuje měsíční report o cenách produktů vybraných vedením. Požádali vás o automatizaci procesu. Po rozhovoru s týmem jste stanovili následující podmínky:

**Parametry reportu:**
- `product_group_id`
- `product`
- `date`

**Pravidla pro parametry:**
1. Parametr může mít nejvýše jednu hodnotu
2. Pokud je parametr prázdný, vrátíme všechny záznamy ze skupiny
3. Předpokládáme, že soubor je vždy správně připravený

**Úkoly:**
1. Načtěte soubor **config.xlsx** pomocí `openpyxl`
2. Připravte podmínky pro filtrování dat z **product_cleaned.csv**
3. Aplikujte filtry na DataFrame
4. Agregujte data pomocí `pivot_table`:
   - index: product, province
   - columns: dates
   - value: průměrná cena produktu
   - nezapomeňte odstranit nuly
5. Uložte výsledek do Excel souboru

**Tipy:**
- Podmínky filtrování můžete uložit do proměnných a pak je kombinovat: `df.loc[var1 & var2]`
- Při ukládání pomocí Pandas dejte pozor na parametr `index` - co se stane když nastavíte `index=False`?

In [65]:
wbc = openpyxl.load_workbook('../Data/config.xlsx')

In [66]:
wbc.sheetnames

['Sheet1', 'Sheet2']

In [67]:
wsc = wbc['Sheet1']

In [68]:
product_group_id = wsc['B2'].value
product_group_id

1

In [69]:
type(product_group_id)

int

In [70]:
product = wsc['B3'].value
product

In [71]:
type(product)

NoneType

In [72]:
date = wsc['B4'].value
date

In [73]:
type(date)

NoneType

In [74]:
maska_group = df['product_group_id'] == product_group_id

In [75]:
maska_product = df['product'] == product
sum(maska_product)

0

In [76]:
product_group_id

1

In [77]:
product

In [78]:
bool(product_group_id)

True

In [79]:
bool(product)

False

In [80]:
maska_group = df['product_group_id'] == (product_group_id if product_group_id else df['product_group_id'])
sum(maska_group)

17119

In [81]:
maska_product = df['product'] == (product if product else df['product'])
sum(maska_product)

128503

In [82]:
maska_date = df['date'] == (date if date else df['date'])
sum(maska_date)

128503

In [83]:
# iba nenulove ceny
maska_ceny = df['value'] > 0
sum(maska_ceny)

87262

In [84]:
df_mod = df.loc[maska_group & maska_product & maska_date & maska_ceny]

In [85]:
df_mod.shape

(9803, 8)

In [86]:
# to iste kompaktnejsie
df_mod = df.loc[
        (df['product_group_id'] == (product_group_id if product_group_id else df['product_group_id'])) & 
        (df['product'] == (product if product else df['product'])) & 
        (df['date'] == (date if date else df['date'])) & 
        (df['value'] > 0)
]

In [87]:
df_mod.columns

Index(['province', 'product_types', 'currency', 'product_group_id',
       'product_line', 'value', 'date', 'product'],
      dtype='object')

In [ ]:
pivot = pd.pivot_table(
    data = df_mod,
    index = ['product', 'province'],
    columns = 'date',
    values = 'value'
)

In [89]:
pivot.shape

(63, 252)

In [90]:
pivot.head(3)

date                                                1999-1  1999-10  1999-11  \
product                          province                                      
30% tomato concentrate - per 1kg GREATER POLAND       7.56     7.02     8.21   
                                 HOLY CROSS           3.81     1.27     1.62   
                                 KUYAVIA-POMERANIA    6.72     6.76     6.55   

date                                                1999-12  1999-2  1999-3  \
product                          province                                     
30% tomato concentrate - per 1kg GREATER POLAND        7.87    7.09    7.32   
                                 HOLY CROSS            0.95    2.67    0.06   
                                 KUYAVIA-POMERANIA     6.90    7.02    6.37   

date                                                1999-4  1999-5  1999-6  \
product                          province                                    
30% tomato concentrate - per 1kg GREATER POLAND       7.84    6.84    7.41   
                                 HOLY CROSS           0.57    2.07    0.01   
                                 KUYAVIA-POMERANIA    6.14    6.39    6.78   

date                                                1999-7  ...  2019-11  \
product                          province                   ...            
30% tomato concentrate - per 1kg GREATER POLAND       7.28  ...     8.33   
                                 HOLY CROSS           2.56  ...      NaN   
                                 KUYAVIA-POMERANIA    7.06  ...      NaN   

date                                                2019-12  2019-2  2019-3  \
product                          province                                     
30% tomato concentrate - per 1kg GREATER POLAND        6.03    5.56    6.78   
                                 HOLY CROSS             NaN     NaN     NaN   
                                 KUYAVIA-POMERANIA      NaN     NaN     NaN   

date                                                2019-4  2019-5  2019-6  \
product                          province                                    
30% tomato concentrate - per 1kg GREATER POLAND       5.67    1.14    3.69   
                                 HOLY CROSS            NaN     NaN     NaN   
                                 KUYAVIA-POMERANIA     NaN     NaN     NaN   

date                                                2019-7  2019-8  2019-9  
product                          province                                   
30% tomato concentrate - per 1kg GREATER POLAND       9.71    5.78    4.96  
                                 HOLY CROSS            NaN     NaN     NaN  
                                 KUYAVIA-POMERANIA     NaN     NaN     NaN  

[3 rows x 252 columns]

In [91]:
pivot.to_excel('../Data/filtered_pivot.xlsx')

In [92]:
# verzia s rokom

In [93]:
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m')

In [94]:
df['year'] = df['date'].dt.year

In [95]:
df_mod = df.loc[
        (df['product_group_id'] == (product_group_id if product_group_id else df['product_group_id'])) & 
        (df['product'] == (product if product else df['product'])) & 
        (df['date'] == (date if date else df['date'])) & 
        (df['value'] > 0)
]

In [96]:
# priemerne ceny za roky
pivot = pd.pivot_table(
    data = df_mod,
    index = ['product', 'province'],
    columns = 'year',
    values = 'value'
).round(2)

In [97]:
pivot.to_excel('../Data/filtered_pivot.xlsx')

---

# Přehled použitých metod a funkcí

| Metoda/Funkce | Popis |
|---------------|-------|
| `openpyxl.Workbook()` | Vytvoří nový Excel sešit |
| `openpyxl.load_workbook(cesta)` | Otevře existující Excel soubor |
| `wb.active` | Vrátí aktivní list sešitu |
| `wb.sheetnames` | Seznam názvů všech listů |
| `wb.create_sheet(name, index)` | Vytvoří nový list s daným názvem na dané pozici |
| `wb.save(cesta)` | Uloží sešit do souboru |
| `wb['NazevListu']` | Vybere list podle názvu |
| `ws['A1']` | Přístup k buňce pomocí adresy |
| `ws['A1'].value` | Přečte hodnotu z buňky |
| `ws.cell(row, column, value)` | Přístup k buňce pomocí čísel řádku a sloupce |
| `ws.append(seznam)` | Přidá řádek dat na konec listu |
| `ws.auto_filter.ref` | Nastaví rozsah pro automatický filtr |
| `ws.auto_filter.add_filter_column(col_id, vals)` | Přidá filtr na sloupec |
| `dataframe_to_rows(df, index, header)` | Převede DataFrame na řádky pro zápis do Excelu |